In [1]:
import sysconfig
import os
import numpy as np
import pandas as pd
import json
import distutils
import scrapy
import requests
import json
import logging
import string
import re
from scrapy.crawler import CrawlerProcess
from scrapy.selector import Selector
from requests import Request

In [2]:
class Stayz_Calendar(scrapy.Item):
    property_id = scrapy.Field()
    collection_date = scrapy.Field()
    calendar_raw = scrapy.Field()

In [3]:
# Setup a pipeline
class JsonWriterPipeline(object):
    def open_spider(self, spider):
        self.file = open('stayz_calendar.jl','w')
        
    def close_spider(self, spider):
        self.file.close()
        
    def processitem(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

In [4]:
!rm stayz_calendar.json

In [5]:
# http://www.thecodeknight.com/post_categories/search/posts/scrapy_python

class StayzSpider(scrapy.Spider):
    name = 'stayz_crawler'
    
    # Full scrape - NSW
    #start_urls = ['https://www.stayz.com.au/accommodation/nsw']
    
    # Forbes - has 3 properties on one listing page
    start_urls = ['https://www.stayz.com.au/accommodation/nsw/explorer-country/forbes']
    
    # Orange  - has 122 properties on 3 listing pages
    #start_urls = ['https://www.stayz.com.au/accommodation/nsw/explorer-country/orange/']
    
    #start_urls = ['https://www.stayz.com.au/accommodation/nsw/explorer-country/mudgee'
    #             ,'https://www.stayz.com.au/accommodation/nsw/explorer-country/bathurst'
    #             ,'https://www.stayz.com.au/accommodation/nsw/north-coast/coffs-harbour'
    #             ,'https://www.stayz.com.au/accommodation/nsw/north-coast/port-macquarie']
    
    #start_urls = ['https://www.stayz.com.au/accommodation/nsw/central-coast/gosford/'
    #              ,'https://www.stayz.com.au/accommodation/nsw/explorer-country/orange/'
    #             ]
    
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, #Used for pipeline 1
        'FEED_FORMAT':'json', # Used for pipeline 2
        'FEED_URI': 'stayz_calendar.json' #Used for pipeline 2
    }
    
    url_pages = []
    
    def parse(self, response):
        
        sel = Selector(response)
               
        urls = sel.xpath('//section[@class="c-search-results__main"]/div/article/div/div/div/h3/a/@href').extract()
                
        for u in urls:
        
            request = scrapy.Request('https://www.stayz.com.au/'+ u, callback=self.parse_results )
            
            yield request
            
        # Get the link to the 'Next' page of listings
        url_pages = sel.xpath('/html/body/div/main/div/section/div/nav/ul/li/a/@href').extract()
        
        if len(url_pages) > 0:
            nextPage = url_pages[-1]
        
            nextPageURL = 'https://www.stayz.com.au/'+ nextPage
        
            print("Next URL: " + str(nextPage))
        
            yield(scrapy.Request(nextPageURL, callback=self.parse))
            
    def strip_whitespace(self, st):
    
        st1 = re.sub('\n','',st.string() )
        st2 = re.sub(' +',' ', st1.string() )
        return st2    
                
    def parse_results(self, response):
        
        # Property ID - unique for each property
        p_id1 = response.selector.xpath('//article/header/ol/li/span/span/text()').extract_first()
        p_id2 = re.sub('\n','',p_id1)
        p_id3 = re.sub(' +',' ', p_id2)
        p_id4 = re.search('\d+',p_id3)
        p_id = p_id4.group(0)

        
        
        # Ratings and reviews
        p_rating = response.selector.xpath('//header[@class="c-reviews__header"]/span/span/span[@class="c-facet__label"]/span/text()').extract_first()
        
        p_r1 = response.selector.xpath('//html/body/main/section/div/article/header/p[2]/span/span[2]/small/text()').extract_first()
        
        
        
        print("Scanning Property ID: " + p_id + " - " + "-" + response.url)
        
        p = Stayz_Calendar()
        p['property_id'] = p_id
        p['calendar_raw'] = 'test'
        
        yield p        


In [6]:
process = CrawlerProcess({ 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)' })

process.crawl(StayzSpider) 
process.start()

2018-01-31 10:16:00 [scrapy.utils.log] INFO: Scrapy 1.5.0 started (bot: scrapybot)
2018-01-31 10:16:00 [scrapy.utils.log] INFO: Versions: lxml 4.1.0.0, libxml2 2.9.4, cssselect 1.0.3, parsel 1.3.1, w3lib 1.18.0, Twisted 16.6.0, Python 3.6.3 |Anaconda, Inc.| (default, Oct  6 2017, 12:04:38) - [GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)], pyOpenSSL 17.2.0 (OpenSSL 1.0.2l  25 May 2017), cryptography 2.0.3, Platform Darwin-17.3.0-x86_64-i386-64bit
2018-01-31 10:16:00 [scrapy.crawler] INFO: Overridden settings: {'FEED_FORMAT': 'json', 'FEED_URI': 'stayz_calendar.json', 'LOG_LEVEL': 30, 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}


Scanning Property ID: 153463 - -https://www.stayz.com.au//accommodation/nsw/explorer-country/forbes/153463
Scanning Property ID: 207273 - -https://www.stayz.com.au//accommodation/nsw/explorer-country/forbes/207273
Scanning Property ID: 147476 - -https://www.stayz.com.au//accommodation/nsw/explorer-country/forbes/147476
